## Part 3 - Train Model

**Connect to Azure ML workspace**

In [10]:
from azureml.core import Workspace

workspace = Workspace.from_config()

**Retrieve dataset and compute targets prepared in earlier steps**

In [3]:
cifar_dataset = workspace.datasets['CIFAR-10 Dataset']
compute_target = workspace.compute_targets['k80cluster']

**Create experiment**

In [11]:
from azureml.core import Experiment

experiment_name = 'objection-classification' 
experiment = Experiment(workspace, name=experiment_name)

**Submit training job**

In [12]:
%pycat train/train.py

In [13]:
from azureml.core.environment import CondaDependencies
from azureml.core import ScriptRunConfig, RunConfiguration
from azureml.core.runconfig import DEFAULT_GPU_IMAGE
from azureml.widgets import RunDetails

# Specify conda packages
conda_dep = CondaDependencies()
conda_dep.add_pip_package('torch==1.4.0')
conda_dep.add_pip_package('torchvision==0.5.0')
conda_dep.add_pip_package('azureml-dataprep[fuse,pandas]==1.4.3')

# Specify run config
run_config = RunConfiguration()
run_config.target = compute_target
run_config.environment.docker.enabled = True
run_config.environment.docker.base_image = DEFAULT_GPU_IMAGE
run_config.environment.python.conda_dependencies = conda_dep

# Specify script arguments
# script_args = ['--data_dir', cifar_dataset.as_named_input('cifardata').as_mount(),
#                 '--output_dir', './outputs', 
#                 '--num_epochs', 10, 
#                 '--batch_size', 16,
#                 '--learning_rate', 0.001, 
#                 '--momentum', 0.9]

# Specify script run config
script_run_config = ScriptRunConfig(source_directory='train', 
                                    script='train.py',
                                    arguments=['--data_dir', cifar_dataset.as_named_input('cifardata').as_mount(),
                                               '--output_dir', './outputs', 
                                               '--num_epochs', 10, 
                                               '--batch_size', 16,
                                               '--learning_rate', 0.001, 
                                               '--momentum', 0.9],
                                    run_config=run_config)

In [14]:
run = experiment.submit(script_run_config)

In [15]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

**Register model**

In [16]:
model = run.register_model(model_name='cifar-classifier', model_path='outputs')